In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,5)
plt.rcParams['axes.grid'] = False
import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})

In [5]:
import time
from data_process import *

## Testing with sinx

In [6]:
train_df, test_df = get_stock_data('sinx', 0.8)
all_actions = {0:'hold', 1:'buy', 2:'sell'}
windows = [20, 40] #20, 40
num_features = len(windows)+1
train_df = create_df(train_df, windows)
price_states_value, fisher_states_value = get_states(train_df)
train_df = create_state_df(train_df, price_states_value, fisher_states_value)
all_states = get_all_states(price_states_value, fisher_states_value)
states_size = len(all_states)
test_df = create_df(test_df, windows)
test_df = create_state_df(test_df, price_states_value, fisher_states_value)


1000

In [7]:
10*2*3.14

62.800000000000004

# Train with single episode

In [8]:
np.random.seed(12)
q = initialize_q_mat(all_states, all_actions)/1e9
print('Initializing q')
print(q[:3])

Initializing q
                   0             1             2
states                                          
000     1.541628e-19  7.400497e-19  2.633150e-19
001     5.337394e-19  1.457496e-20  9.187470e-19
002     9.007149e-19  3.342143e-20  9.569493e-19


In [25]:
train_data = train_df[['Normalized_Close', 'state']]
q, train_actions_history, train_returns_since_entry, model = train_q_learning(train_data, q, len(all_states), use_nn=True, alpha=0.8, gamma=0.95, episodes=100, theta=1)

  0%|          | 0/100 [04:29<?, ?it/s]

KeyboardInterrupt



In [ ]:
visualize_results(train_actions_history, train_returns_since_entry)
get_invested_capital(train_actions_history, train_returns_since_entry)
print('base return/invest ratio {}'.format(get_base_return(np.array(train_data))))

## Test evaluation

In [ ]:
test_data = np.array(test_df[['Normalized_Close', 'state']])
test_actions_history, test_returns_since_entry = eval_q_learning(test_data, q)

In [10]:
visualize_results(test_actions_history, test_returns_since_entry)
get_invested_capital(test_actions_history, test_returns_since_entry)
# print('invested capital {}, return/invest ratio {}'.format(invested_capital, return_invest_ratio))
print('base return/invest ratio {}'.format(get_base_return(test_data)))




NameError: name 'test_actions_history' is not defined

In [15]:
import gym
import numpy as np
from keras.models import Sequential
from keras.layers import InputLayer
from keras.layers import Dense

In [16]:
env = gym.make('FrozenLake-v1')

In [17]:
discount_factor = 0.95
eps = 0.5
eps_decay_factor = 0.999
num_episodes=500

In [18]:
env.observation_space.n

16

In [19]:
model = Sequential()
model.add(InputLayer(batch_input_shape=(1, env.observation_space.n)))
model.add(Dense(20, activation='relu'))
model.add(Dense(env.action_space.n, activation='linear'))
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [21]:
for i in tqdm(range(num_episodes)):
    state = env.reset()
    eps *= eps_decay_factor
    done = False
    while not done:
        if np.random.random() < eps:
            action = np.random.randint(0, env.action_space.n)
        else:
            print(env.observation_space.n)
            print(state)
            print("*"*100)
            action = np.argmax(
              model.predict(np.identity(env.observation_space.n)[state:state + 1]))
        new_state, reward, done, _ = env.step(action)
        target = reward +discount_factor *np.max(
              model.predict(
                np.identity(env.observation_space.n)[new_state:new_state + 1]))
        target_vector = model.predict(
          np.identity(env.observation_space.n)[state:state + 1])[0]
        target_vector[action] = target
        model.fit(
          np.identity(env.observation_space.n)[state:state + 1], target_vector.reshape(-1, env.action_space.n),
          epochs=1, verbose=0)
        state = new_state

  0%|          | 0/500 [00:00<?, ?it/s]

16
0
****************************************************************************************************
16
4
****************************************************************************************************


  0%|          | 2/500 [00:00<02:54,  2.86it/s]

16
0
****************************************************************************************************
16
0
****************************************************************************************************
16
0
****************************************************************************************************
16
4
****************************************************************************************************


  1%|          | 3/500 [00:01<05:44,  1.44it/s]

...

16
0
****************************************************************************************************
16
0
****************************************************************************************************
16
0
****************************************************************************************************
16
1
****************************************************************************************************


  1%|          | 5/500 [00:03<05:33,  1.48it/s]

16
4
****************************************************************************************************
16
0
****************************************************************************************************
16
0
****************************************************************************************************
16
0
****************************************************************************************************
16
0
****************************************************************************************************
16
0
****************************************************************************************************
16
0
****************************************************************************************************


  1%|▏         | 7/500 [00:04<05:51,  1.40it/s]


KeyboardInterrupt: 